### Aprendizado Ensemble - Florestas Aleatórias

In [1]:
# bibliotecas utilizadas no desenvolvimento do projeto
from collections import Counter, defaultdict, OrderedDict
from functools import partial
import math, random
import pandas as pd
from csv import reader
import numpy as np
from random import seed
from random import randrange
from graphviz import Digraph

In [2]:
def build_dataset(path_file): 
    dataset = list()
    with open(path_file, 'r') as file:
        csv_reader = reader(file)
        rows = [r for r in csv_reader]
    # capturando os cabeçalhos
    headers = rows[0]
    
    remove_row_with_values_missing(rows, headers)
    
    for row in rows[1:len(rows)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        dataset.append(data_tuple)
    return dataset

# TODO: devemos remover linhas com dados faltando?
def remove_row_with_values_missing(rows, headers):
    quant = 0
    for row in rows[1:len(rows)]:
        for header_index in range(len(headers)):
            if(row[header_index] is None or row[header_index] == ""):
                quant = quant + 1
    

# construindo a base de dados a partir do arquivo fornecido
dataset = build_dataset('datasets/benchmark.csv')

1. Desenvolvimento do algoritmo de indução de uma árvore de decisão, usando como critério de seleção de atributos para divisão de nós o **Ganho de Informação (baseado no conceito de entropia)**. **Tratando tanto atributos categóricos quanto numéricos.**

In [3]:

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values()) # retorna a media dos sub-conjuntos resultantes

def gain(class_entropy, mean_entropy):
    return class_entropy - mean_entropy;

def class_entropy(inputs):
    """retorna a entropia sobre a classe"""
    class_values = []
    for item in inputs:
        class_values.append(item[1]) # suponto a definição atual dos dados!
    probabilities = class_probabilities(class_values)
    class_entropy = entropy(probabilities)
    return class_entropy

def diff_class(inputs):
    classes = []
    for current_class in list(set([label for item, label in inputs if label])):
        classes.append({'class': current_class, 'amount': 0})
    for current_class in [label for item, label in inputs if label]:
        for my_class in classes:
            if(my_class['class']  == current_class):
                my_class['amount'] = my_class['amount'] + 1
    return classes

def highest_class(classes):
    my_class = classes[0]
    for current_class in classes:
        if(current_class['amount'] > my_class['amount']):
            my_class = current_class
    return my_class
    

def build_tree(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
              
    # obtendo as classes
    # [(classe, quantidade)]
    classes = diff_class(inputs)
    
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    # verificando a homogeneidade do cojunto
    if(len(classes) == 1):
        return classes[0]['class']

    if not split_candidates:          # if no split candidates left
        return highest_class(classes) # return the majority leaf

    # otherwise, split on the best attribute
    
    best_gain = 0
    best_attribute = None
    for candidate in split_candidates:
        mean_entropy = partition_entropy_by(inputs, candidate)
        print('[INFO] Atributo: ', candidate, '- entropia média: ', mean_entropy)
        gain_result = gain(class_entropy(inputs), mean_entropy)
        print('[INFO] Atributo: ', candidate, '- ganho: ', gain_result)
        if(gain_result > best_gain):
            best_gain = gain(class_entropy(inputs), mean_entropy)
            best_attribute = candidate
        print('--')
    
    print('----------------------------------------')
    print('[INFO] best_attribute: ', best_attribute)
    print('----------------------------------------')
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":
    # construindo a árvore sobre os dados construindos anteriormente
    tree = build_tree(dataset)
    print('\n')
    print('----------------------------------------')
    print("[INFO] Árvore construida:")
    print('----------------------------------------')
    print(tree)

[INFO] Atributo:  Tempo - entropia média:  0.6935361388961919
[INFO] Atributo:  Tempo - ganho:  0.246749819774439
--
[INFO] Atributo:  Temperatura - entropia média:  0.9110633930116763
[INFO] Atributo:  Temperatura - ganho:  0.029222565658954647
--
[INFO] Atributo:  Umidade - entropia média:  0.7884504573082896
[INFO] Atributo:  Umidade - ganho:  0.15183550136234136
--
[INFO] Atributo:  Ventoso - entropia média:  0.8921589282623617
[INFO] Atributo:  Ventoso - ganho:  0.04812703040826927
--
----------------------------------------
[INFO] best_attribute:  Tempo
----------------------------------------
[INFO] Atributo:  Temperatura - entropia média:  0.4
[INFO] Atributo:  Temperatura - ganho:  0.5709505944546686
--
[INFO] Atributo:  Umidade - entropia média:  0.0
[INFO] Atributo:  Umidade - ganho:  0.9709505944546686
--
[INFO] Atributo:  Ventoso - entropia média:  0.9509775004326938
[INFO] Atributo:  Ventoso - ganho:  0.01997309402197478
--
----------------------------------------
[INFO] 

In [4]:
g = Digraph('G', filename='tree.gv')

g.edge('Tempo, ganho 0.970', 'Umidade', label='Ensolarado, ganho: 0.970')
g.edge('Umidade', 'Não', label='Alta')
g.edge('Umidade', 'Sim', label='Normal')
g.edge('Tempo, ganho 0.970', 'Sim', label='Nublado')
g.edge('Tempo, ganho 0.970', 'Ventoso', label='Nublado, ganho: 0.970')
g.edge('Ventoso', 'Sim', label='Falso')
g.edge('Ventoso', 'Nao', label='Verdadeiro')

#g.view()

2. Uma função para percorrer a árvore de decisão treinada e realizar a classificação de uma nova instância (do conjunto de teste);

In [5]:
def classify(tree, classes, input):
    """classificar a entrada usando a árvore de decisão fornecida"""

    # se este é um nó folha, retorna seu valor
    if tree in classes:
        return tree

    # caso contrário, encontre a subárvore correta
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # Nenhum se a entrada estiver faltando atributo

    if subtree_key not in subtree_dict: # se não houver subárvore para a chave
        subtree_key = None              # vamos usar a subárvore não

    subtree = subtree_dict[subtree_key] # escolha a subárvore apropriada
    return classify(subtree, classes, input) # e usá-lo para classificar a entrada


# obtendo as classes possiveis para a àrvore
classes = list(set([label for item, label in dataset if label]))

print('\n')
print('[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:')

print('Nublado / Quente / Normal / Falso ->', classify(tree, classes,
    { 'Tempo' : 'Nublado',
      'Temperatura' : 'Quente',
      'Umidade' : 'Normal',
      'Ventoso' : 'Falso'} ))
print('Chuvoso / Amena / Alta / Verdadeiro ->', classify(tree, classes,
    { 'Tempo' : 'Chuvoso',
      'Temperatura' : 'Amena',
      'Umidade' : 'Alta',
      'Ventoso' : 'Verdadeiro'} ))



[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:
Nublado / Quente / Normal / Falso -> Sim
Chuvoso / Amena / Alta / Verdadeiro -> Nao


3. O mecanismo de bootstrap (amostragem com reposição) para geração de subconjuntos a partir do conjunto de dados de treinamento originais. Cada bootstrap será utilizado para o treinamento de uma árvore no aprendizado ensemble

In [6]:
def bootstrap(dataset):
    bootstrap = dataset.sample(n = len(dataset), replace = True)
    return bootstrap

result_bootstrap = bootstrap(pd.read_csv('datasets/voting.csv'))

In [7]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

result_cross = cross_validation_split(result_bootstrap.values.tolist(), 10)

In [8]:
# TODO: DIVISÃO TREINO E TESTE
def train_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        return train_set

def test_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        test_set = list(fold)
        return test_set


# Não está retendo os dados do banco de bootstrap, é os mesmos dados, mas ainda tá flutuando e gerando listas diferentes, temos que discutir se isso tem algum problema 


test_set = test_split(result_cross, 10) 
print('[INFO] definindo folds para teste')

print('\n[INFO] definindo folds para treino')
train_set = train_split(result_cross, 10)

[INFO] definindo folds para teste

[INFO] definindo folds para treino


In [9]:
def build_from_split(headers, dataset_from_bootstrap): 
    new_dataset = list()    
    for row in dataset_from_bootstrap[0:len(dataset_from_bootstrap)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        new_dataset.append(data_tuple)
    return new_dataset

# obtebdo os cabeçalhos
headers = result_bootstrap.columns.tolist()
#print(build_tree(build_from_bootstrap(headers, result_bootstrap.values.tolist())))


In [10]:
def build_pre_dataset(old_dataset):
    new_dataset = list()
    for i in old_dataset:
        for j in i:
            new_dataset.append(j)
    return new_dataset

In [11]:
# construindo a árvore a partir do split vindo do cross
new_train = build_pre_dataset(train_set) # treino

new_tree_from_train = build_tree(build_from_split(headers, new_train))

# construindo a árvore a partir do split vindo do cross
new_test = build_pre_dataset(test_set) # teste
        
classes = list(set([label for item, label in build_from_split(headers, new_train) if label]))
     
classe_teste = list()
preditos_teste = list()
for current in build_from_split(headers, new_test):
    preditos_teste.append(current[1])
    classe_teste.append(classify(new_tree_from_train, classes, current[0]))
  
#print('\n[INFO] resultado real dos dados de teste: ')
#print(classe_teste)
#print('\n[INFO] resultado preditos esperado: ')
#print(preditos_teste)

[INFO] Atributo:  handicapped-infants - entropia média:  0.8756399866371883
[INFO] Atributo:  handicapped-infants - ganho:  0.09561211140101122
--
[INFO] Atributo:  water-project-cost-sharing - entropia média:  0.9703196232969301
[INFO] Atributo:  water-project-cost-sharing - ganho:  0.0009324747412694512
--
[INFO] Atributo:  adoption-of-the-budget-resolution - entropia média:  0.5525901064630407
[INFO] Atributo:  adoption-of-the-budget-resolution - ganho:  0.4186619915751588
--
[INFO] Atributo:  physician-fee-freeze - entropia média:  0.2302214704124224
[INFO] Atributo:  physician-fee-freeze - ganho:  0.7410306276257772
--
[INFO] Atributo:  el-salvador-aid - entropia média:  0.5899972294251535
[INFO] Atributo:  el-salvador-aid - ganho:  0.381254868613046
--
[INFO] Atributo:  religious-groups-in-schools - entropia média:  0.8669277962958974
[INFO] Atributo:  religious-groups-in-schools - ganho:  0.10432430174230212
--
[INFO] Atributo:  anti-satellite-test-ban - entropia média:  0.83025

[INFO] Atributo:  aid-to-nicaraguan-contras - ganho:  0.010035696300314534
--
[INFO] Atributo:  mx-missile - entropia média:  0.5431983605503997
[INFO] Atributo:  mx-missile - ganho:  0.04847441803192776
--
[INFO] Atributo:  immigration - entropia média:  0.5512567296596851
[INFO] Atributo:  immigration - ganho:  0.04041604892264239
--
[INFO] Atributo:  synfuels-corporation-cutback - entropia média:  0.3284093786780839
[INFO] Atributo:  synfuels-corporation-cutback - ganho:  0.2632633999042436
--
[INFO] Atributo:  education-spending - entropia média:  0.38342707289637756
[INFO] Atributo:  education-spending - ganho:  0.2082457056859499
--
[INFO] Atributo:  superfund-right-to-sue - entropia média:  0.48245466118081465
[INFO] Atributo:  superfund-right-to-sue - ganho:  0.10921811740151283
--
[INFO] Atributo:  crime - entropia média:  0.5304474362858254
[INFO] Atributo:  crime - ganho:  0.06122534229650212
--
[INFO] Atributo:  duty-free-exports - entropia média:  0.5639829886771994
[INFO]

KeyError: None

In [ ]:
matrix = list()
accuracy = list()
recall = list()
precision = list()
score = list()

def classification_report(classe_teste, preditos_teste):
    true_class = 'democrat'
    negative_class = 'republican'
    # declarando a frêquencia
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for (indice, v_real) in enumerate(classe_teste):
        v_predito = preditos_teste[indice]
        # se trata de um valor real da classe positiva
        if v_real == true_class:
            tp += 1 if v_predito == v_real else 0
            fp += 1 if v_predito != v_real else 0
        else:
            tn += 1 if v_predito == v_real else 0
            fn += 1 if v_predito != v_real else 0
                
    # calculando os valores de frêquencia
    accuracy.append((tp+tn)/(tp+fp+tn+fn))
    recall.append(tp/(tp+fn))
    precision.append(tp/(tp+fp))
    score.append(2*tp/(2*tp+fp+fn))
    
    # desenhando a matriz de confusão
    print(np.array([
        [ tp, fp ],
        [ fn, tn ]
    ]))
    
classification_report(classe_teste, preditos_teste)
print('')
print('Média da acurácia: %.2f%%' % (sum(accuracy)/float(len(accuracy))))
print('Média da Recall: %.2f%%' % (sum(recall)/float(len(recall))))
print('Média da Precisão: %.2f%%' % (sum(precision)/float(len(precision))))
print('Média da f-score: %.2f%%' % (sum(score)/float(len(score))))
print('')
print("Desvio padrão da acurácia: {:.2f}%".format(np.std(accuracy)*100))
print("Desvio padrão da Recall: {:.2f}%".format(np.std(recall)*100))
print("Desvio padrão da Precisão: {:.2f}%".format(np.std(precision)*100))
print("Desvio padrão da f-score: {:.2f}%".format(np.std(score)*100))

In [ ]:
# TODO: FLORESTAS ALEATÓRIAS

dados_originais = pd.read_csv('datasets/voting.csv')

n_trees = 15
my_tress = list()
my_cases_for_test = list()
for n in range(n_trees):
    n_bootstrap = bootstrap(dados_originais)       
    n_cross = cross_validation_split(n_bootstrap.values.tolist(), 10)
    my_cases_for_test.append(build_pre_dataset(test_split(n_cross, 10)))
    train_set = train_split(n_cross, 10)
    new_tree_from_train = build_tree(build_from_split(headers, new_train))
    my_tress.append(new_tree_from_train)

In [ ]:
def forest_classify(trees, classes, input):
    votes = [classify(tree, classes, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

classe_teste = list()
preditos_teste = list()
for current in build_from_split(headers, my_cases_for_test[0]):
    preditos_teste.append(current[1])
    classe_teste.append(forest_classify(my_tress, classes, current[0]))

In [ ]:
classification_report(classe_teste, preditos_teste)
print('')
print('Média da acurácia: %.2f%%' % (sum(accuracy)/float(len(accuracy))))
print('Média da Recall: %.2f%%' % (sum(recall)/float(len(recall))))
print('Média da Precisão: %.2f%%' % (sum(precision)/float(len(precision))))
print('Média da f-score: %.2f%%' % (sum(score)/float(len(score))))
print('')
print("Desvio padrão da acurácia: {:.2f}%".format(np.std(accuracy)*100))
print("Desvio padrão da Recall: {:.2f}%".format(np.std(recall)*100))
print("Desvio padrão da Precisão: {:.2f}%".format(np.std(precision)*100))
print("Desvio padrão da f-score: {:.2f}%".format(np.std(score)*100))

4. O mecanismo de amostragem de m atributos a cada divisão de nó, a partir dos quais será selecionado o melhor atributo de acordo com o critério de Ganho de Informação

5. O treinamento de um ensemble de árvores de decisão, adotando os mecanismos de bootstrap e seleção de atributos com amostragem, como mencionados acima


6. O mecanismo de votação majoritária entre as múltiplas árvores de decisão no ensemble, para classificação de novas instâncias utilizando o modelo de Florestas Aleatórias

7. A técnica de validação cruzada (cross-validation) estratificada, para avaliar poder de generalização do modelo e a variação de desempenho de acordo com diferentes valores para os parâmetros do algoritmo (ex., número de árvores no ensemble)

8. Avaliação do impacto do número de árvores no desempenho do ensemble